## 1. Introduction

### a. Project's goal

### b. Project's constraints

## 2. Computer Vision

### a. Scene initialization

### b. Detection of obstacles, Thymio and goal point

## 3. Global Navigation

In this part we'll describe three important steps. The first one is the visibility matrix, it helps the robot see its surroundings by figuring out what’s in its line of sight. The second one is the path planning, where the robot figures out the best route to reach its goal while avoiding obstacles. Then we have the robot's actuation which is how the robot actually moves along the planned path.  

### a. Visibility matrix


We chose the visibility graph approach because it’s simple and finds the shortest path while avoiding obstacles. It’s a practical and efficient method that works well for this kind of navigation problems.

To implement the visibility matrix we used the Shapely library, we imported LineString and Polygon. LineString is used to represent the line segments between pairs of corners while Polygon is used to represents the abstacles as polygons. Those allow us to perform geometric operations such as testing whether a line intersects the obstacle, touches its boundary, or is fully contained within it.

In [ ]:
from shapely.geometry import LineString, Polygon

def compute_visibility_matrix(start,end,obstacles):
    """
    Compute a visibility matrix

    input:
        start point : start position of the robot
        end point : end point; where the robot needs to stop
        obstacles: List of obstacles, each as a list of extended 
                   corner coordinates [[(x1, y1), ...], ...]
    
    output:
        Visibility_mat : (N x N) Visibility_mat(i,j) is equal to 1 if a path exist between the ith corner
                            and the jth corner, if not  it's equal to 0.
        corners: (1 X N) a list of corners
    """

    # Convert start and end points to single-point obstacles
    start_obstacle = [start]
    end_obstacle = [end]

    # Add start and end points to the obstacles list
    obstacles = [start_obstacle, end_obstacle] + obstacles

    # Flatten the list of obstacles to get all corners
    corners = [corner for obstacle in obstacles for corner in obstacle]
    corners = np.array(corners)
    N = len(corners)
    matrix = np.ones((N, N), dtype=int)

    # Convert obstacles to polygons

    obstacle_polygons = []
    for obs in obstacles:
        if len(obs) >= 3:
            obstacle_polygons.append(Polygon(obs))
    obstacle_polygons = np.array(obstacle_polygons)

    # Map corners to their respective obstacle indices
    corner_to_obstacle = {}
    for obs_idx, obstacle in enumerate(obstacles):
        for corner in obstacle:
            corner_to_obstacle[tuple(corner)] = (obs_idx, obstacle)

    for i in range(N):
        for j in range(N):
            if i == j:
                continue

            line = LineString([corners[i], corners[j]])
            # Check if corners[i] and corners[j] belong to the same obstacle
            same_obstacle = (
                tuple(corners[i]) in corner_to_obstacle 
                and tuple(corners[j]) in corner_to_obstacle 
                and corner_to_obstacle[tuple(corners[i])][0] == corner_to_obstacle[tuple(corners[j])][0]
            )

            # If they are from the same obstacle
            if same_obstacle:
                # Retrieve the obstacle corners
                obstacle_corners = corner_to_obstacle[tuple(corners[i])][1]
                # Check if they are adjacent corners (only adjacent corners are visible)
                if not are_adjacent_corners(corners[i], corners[j], obstacle_corners):
                    matrix[i, j] = 0

            else:
                for poly in obstacle_polygons:
                    if line.intersects(poly) and not line.touches(poly.boundary):
                        matrix[i, j] = 0
                        break


    return matrix, corners
    

def are_adjacent_corners(corner1, corner2, obstacle_corners):
    """
    Check if two corners are adjacent in the obstacle.

    input:
        corner1: First corner (x, y)
        corner2: Second corner (x, y)
        obstacle_corners: List of corners of the obstacle
    
    output:
        True if the corners are adjacent, False otherwise
    """

    n = len(obstacle_corners)
    for i in range(n):
        if (np.array_equal(obstacle_corners[i], corner1) and 
           (np.array_equal(obstacle_corners[(i + 1) % n], corner2) or 
           np.array_equal(obstacle_corners[(i - 1) % n], corner2))):
            return True
    return False

Example:

## PHOTO LIGNE POSSIBLES

### b. Path planning

We chose to implement the A* algorithm for the path planning because it's very efficient, it also reduces the unnecessary exploration by using the heuristic function.

The algorithm starts by initializing a priority queue with the start node, where each node is scored based on the total cost (f_cost = g_cost + heuristic). At each step, the node with the lowest f_cost is processed. If the goal is reached, the path is reconstructed by tracing back through the came_from dictionary.

For each neighbor of the current node, the algorithm calculates the cost (g_cost). If this cost is lower than a previously recorded cost, the neighbor is updated in the priority queue. The process continues until the goal is reached or no valid path is found, ensuring an optimal path with minimal computation.

In [ ]:
def heuristic(p1, p2):
    # Implement the Manhattan distance heuristic
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])



def a_star_search(points,ex_path):
    """
    A* 

    input:
        start
        end 
        points: N x 2 array (N = (# of corners) ), where:
            - The remaining rows contain the coordinates of the extended corners.
        ex_path: N X N, is equal to 1 if two corners are directly connected 
              by a line that does not cross any obstacle, and 0 otherwise.

    output: 
        shortest_path: M X 2 (M = # of corners in the shortest path), A list 
            of corner indices representing the shortest path from start to goal.

    Note: This implementation is heavily inspired by the A* algorithm provided in 
      the solution to the 5th exercise session of the Mobile Robotics course.
      
    """

    
    # Initialize the open set as a priority queue and add the start node
    open_set = []
    start_index = 0
    start = points[0,:]
    goal_index = 1
    goal = points[1,:]

    N = points.shape[0]
    distance_matrix = np.zeros((N, N))
    for i in range(N):
         for j in range(N):
              #calculate the distance between two corners
              distance_matrix[i,j] = norm(points[i,:]-points[j,])
    

    heappush(open_set, (heuristic(start, goal), 0, start_index))  # (f_cost, g_cost, position)

    # Initialize the came_from dictionary
    came_from = {}
    # Initialize g_costs dictionary with default value of infinity and set g_costs[start] = 0
    g_costs = {start_index: 0}
    # Initialize the explored set
    explored = set()
    operation_count = 0

    while open_set:
        # Pop the node with the lowest f_cost from the open set
        current_f_cost, current_g_cost, current_index = heappop(open_set)
        # Add the current node to the explored set
        explored.add(current_index)

        # For directly reconstruct path
        if current_index == goal_index:
            break

        # Get the neighbors of the current node 
        index_vector = ex_path[current_index,:]
        neighbors_index = np.nonzero(index_vector)[0]

        for index in neighbors_index:
             if  index not in explored:
                    # Calculate tentative_g_cost
                    tentative_g_cost = current_g_cost + distance_matrix[current_index,index]

                    # If this path to neighbor is better than any previous one
                    if index not in g_costs or tentative_g_cost < g_costs[index]:
                        # Update came_from, g_costs, and f_cost
                        came_from[index] = current_index
                        g_costs[index] = tentative_g_cost
                        f_cost = tentative_g_cost + heuristic(points[index,:], goal)
                        
                        # Add neighbor to open set
                        heappush(open_set, (f_cost, tentative_g_cost, index))
                        operation_count += 1

    # Reconstruct path
    if current_index == goal_index:
        path = []
        while current_index in came_from:
            path.append(points[current_index,:])
            current_index = came_from[current_index]
        path.append(start)
        np_path = np.array(path)
        return np_path[::-1]
    else:
        # If we reach here, no path was found
        return None

Example:

## Photo qui montre le chemin optimal

### c. Robot's actuation

## 4 Local Navigation

### a. Obstacle avoidance (Artificial Neural Network)

### b. Completion of the obstacle avoidance

## 5. Position Estimation: Kalman Filter

### a. Empirical tests

### b. Kalman filtering theory

## 6. Video Demonstrations

### a. Global example

### b. Specific cases

#### i. Obstacle avoidance

#### ii. Kidnapping

#### iii. Change in goal's position

#### iv. Hidden Camera